<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_opsmps3copy.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Agnostic App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.

In this notebook we will modify the opensees-mp-s3 for OpenSeesMP or OpenSeesSP, etc.

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
pwd

'/home/jupyter/MyData/_ToCommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks'

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


In [3]:
import json

In [4]:
# Make these True once you are done validating
do_makeAppFiles = True
do_upload = True
do_register = True
do_submitJob = True

---
## Connect to Tapis

In [5]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-19T05:35:46+00:00
 Token expires in: 3:12:14.755771
-- LOG IN SUCCESSFUL! --


---
### Get username

In [6]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [7]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

username silvia


---
## Configure App

In [8]:
app_id = 'agnostic-app-test'
app_version = '0.0.7'



In [9]:
app_description = 'Run any Executable'

app_system_id="designsafe.storage.default"
app_path = f"{username}/apps/{app_id}/{app_version}"
container_filename = f'{app_id}.zip'

### Define folder structure: app_id/version

In [10]:
app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [11]:
app_folder = os.path.abspath(os.path.expanduser(app_folderName))
os.makedirs(app_folder, exist_ok=True)
print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

app_folder: /home/jupyter/MyData/myAuthoredTapisApps/agnostic-app-test/0.0.7
 exists: True


---
## Create the App Files

A Tapis app needs the following **core files**:

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [12]:
# let's collect the list of files we are creating as we will need this list in upload
appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [13]:
thisFilename = 'ReadMe.MD'
if do_makeAppFiles:
    thisText = """\
    # making an application-agnostic app
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [14]:
thisFilename = 'app.json'
if do_makeAppFiles:
    thisText = """\
    {
      "id": "__app_id__",
      "version": "__app_version__",
      "description": "__app_description__",
      "owner": "${apiUserId}",
      "enabled": true,
      "runtime": "ZIP",
      "runtimeVersion": null,
      "runtimeOptions": null,
      "containerImage": "__container_filename__",
      "jobType": "BATCH",
      "maxJobs": -1,
      "maxJobsPerUser": -1,
      "strictFileInputs": true,
      "jobAttributes": {
        "execSystemConstraints": null,
        "execSystemId": "stampede3",
        "execSystemExecDir": "${JobWorkingDir}",
        "execSystemInputDir": "${JobWorkingDir}",
        "execSystemOutputDir": "${JobWorkingDir}",
        "execSystemLogicalQueue": "skx",
        "archiveSystemId": "stampede3",
        "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
        "archiveOnAppError": true,
        "isMpi": false,
        "mpiCmd": null,
        "parameterSet": {
          "appArgs": [
            {
              "name": "Main Program",
              "description": "Choose the binary Executable to use.",
              "arg": "OpenSeesMP",
              "inputMode": "REQUIRED",
                "notes": {
                    "enum_values": [
                        {
                            "OpenSeesMP": "OpenSeesMP"
                        },
                        {
                            "OpenSeesSP": "OpenSeesSP"
                        },
                        {
                            "python": "python"
                        }
                    ]
                }
            },
            {
              "name": "Main Script",
              "description": "The filename only of the input script to submit to executable. This file should reside in the Input Directory specified.",
              "arg": null,
              "inputMode": "REQUIRED",
              "notes": {
                "inputType": "fileInput"
              }
            },
            {
              "name": "CommandLine Arguments",
              "description": "Command-Line Arguments (Optional)",
              "arg": null,
              "inputMode": "INCLUDE_ON_DEMAND",
              "notes": {
              }
            }
            
          ],
          "containerArgs": [],
          "schedulerOptions": [
            {
              "name": "OpenSees TACC Scheduler Profile",
              "description": "Scheduler profile for the default version of OpenSees",
              "inputMode": "FIXED",
              "arg": "--tapis-profile OpenSees_default",
              "notes": {
                "isHidden": true
              }
            }
          ],
          "envVariables": [
                  { "key": "zipFileIn", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "zipFolderOut", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" },
                  { "key": "requirementFile", "value": "", "inputMode": "INCLUDE_BY_DEFAULT" }
             ],
          "archiveFilter": {
            "includes": [],
            "excludes": [],
            "includeLaunchFiles": true
          }
        },
        "fileInputs": [
          {
            "name": "Input Directory",
            "inputMode": "REQUIRED",
            "sourceUrl": null,
            "targetPath": "inputDirectory",
            "envKey": "inputDirectory",
            "description": "Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP",
            "notes": {
              "selectionMode": "directory"
            }
          }
        ],
        "fileInputArrays": [],
        "nodeCount": 1,
        "coresPerNode": 48,
        "memoryMB": 192000,
        "maxMinutes": 120,
        "subscriptions": [],
        "tags": []
      },
      "tags": [
        "portalName: DesignSafe",
        "portalName: CEP"
      ],
      "notes": {
        "label": "__app_id__",
        "helpUrl": "",
        "hideNodeCountAndCoresPerNode": false,
        "isInteractive": false,
        "icon": "OpenSees",
        "category": "Simulation"
      }
    }
    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    thisText = thisText.replace("__app_description__", app_description)
    thisText = thisText.replace("__container_filename__", f"tapis://{app_system_id}/{app_path}/{container_filename}")
    
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [15]:
thisFilename = 'profile.json'
if do_makeAppFiles:
    # thisText = """\
    # {
    #     "name": "AgnosticApp",
    #     "description": "Modules to load",
    #     "moduleLoads": [ ],
    #     "hiddenOptions": [
    #         "MEM"
    #     ]
    # }
    # """    
    thisText = """\
    {
        "name": "AgnosticApp",
        "description": "Modules to load",
        "moduleLoads": [
            {
                "modulesToLoad": [
                    "hdf5/1.14.4",
                    "opensees"
                ],
                "moduleLoadCommand": "module load"
            }
        ],
        "hiddenOptions": [
            "MEM"
        ]
    }
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [16]:
import textwrap, time
from zipfile import ZipFile, ZIP_DEFLATED, ZipInfo

thisFilename_sh = "tapisjob_app.sh"
thisFilename = container_filename  # existing var

if do_makeAppFiles:
    bash_script = textwrap.dedent("""\
        #!/bin/bash
        set -euo pipefail
        set -x

        # ---- required args ----
        BINARYNAME="${1:?missing binary name}"
        INPUTSCRIPT0="${2:?missing input script}"
        shift 2

        # ---- optional env with safe defaults ----
        inputDirectory="${inputDirectory:?inputDirectory not set}"
        zipFileIn="${zipFileIn:-}"
        zipFolderOut="${zipFolderOut:-}"

        INPUTSCRIPT="${INPUTSCRIPT0##*/}"
        echo "INPUTSCRIPT: $INPUTSCRIPT"
        echo "ARGS: $*"

        cd -- "$inputDirectory"

        # ---- expand input ZIP if provided ----
        if [[ -n "$zipFileIn" && -f "$zipFileIn" ]]; then
          echo "Unzipping $zipFileIn ..."
          unzip -o -q "$zipFileIn"
        fi

        # ---- modules for OpenSees family ----
        case "$BINARYNAME" in
          OpenSees|OpenSeesMP|OpenSeesSP)
            module load hdf5/1.14.4
            module load opensees
            module list || true
            echo "Loaded OpenSees modules"
            ;;
        esac

        # ---- choose launcher ----
        if [[ "$BINARYNAME" == "OpenSees" ]]; then
          LAUNCH=()        # direct run for sequential
        else
          LAUNCH=(ibrun)   # MPI/SP use ibrun on TACC
        fi

        echo "Running: ${LAUNCH[*]} $BINARYNAME $INPUTSCRIPT $*"
        if ! "${LAUNCH[@]}" "$BINARYNAME" "$INPUTSCRIPT" "$@"; then
          rc=$?
          echo "Program exited with error status: $rc" >&2
          exit "$rc"
        fi

        # ---- optional re-pack an output folder ----
        if [[ -n "$zipFolderOut" && -d "$zipFolderOut" ]]; then
          echo "Zipping output folder $zipFolderOut -> ${zipFolderOut}.zip"
          zip -r -q "${zipFolderOut}.zip" "$zipFolderOut"
          rm -rf -- "$zipFolderOut"
        fi
    """)


    zip_path = os.path.join(app_folder, thisFilename)
    print('zip_path',zip_path)

    # write with executable permission inside the ZIP
    zi = ZipInfo(thisFilename_sh)
    zi.date_time = time.localtime(time.time())[:6]
    zi.compress_type = ZIP_DEFLATED
    zi.external_attr = 0o100755 << 16   # -rwxr-xr-x on a regular file
    
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
        z.writestr(zi, bash_script)
        
appFilesList.append(thisFilename)

zip_path /home/jupyter/MyData/myAuthoredTapisApps/agnostic-app-test/0.0.7/agnostic-app-test.zip


#### File Check
Look at the files we have written and check for typos or formatting errors.

In [17]:
print(appFilesList)
OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

['ReadMe.MD', 'app.json', 'profile.json', 'agnostic-app-test.zip']


---
## Validate App Locally

In [18]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [19]:
# Run validation
validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
if not validation:
    print('Validation Failed: stopping here!!!!')
    a = 3/0

🔍 Validating app folder: /home/jupyter/MyData/myAuthoredTapisApps/agnostic-app-test/0.0.7

✅ All required files are present.

📄 App ID: agnostic-app-test
📄 App Name: AgnosticApp
📄 Version: 0.0.7
🔧 Parameters: []
📦 Inputs: []
📤 Outputs: []

App Keys: ['id', 'version', 'description', 'owner', 'enabled', 'runtime', 'runtimeVersion', 'runtimeOptions', 'containerImage', 'jobType', 'maxJobs', 'maxJobsPerUser', 'strictFileInputs', 'jobAttributes', 'tags', 'notes', 'name', 'moduleLoads', 'hiddenOptions']

✅ Basic validation complete. App folder looks good!


---
## Deploy the App

### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [20]:
if do_upload:
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

app_path silvia/apps/agnostic-app-test/0.0.7


### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [21]:
if do_upload:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [22]:
if do_upload:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    appfiles = t.files.listFiles(systemId=app_system_id, path=app_path)
    for thisF in appfiles:
        print(thisF)
        print('')

app_system_id: designsafe.storage.default
app_path: silvia/apps/agnostic-app-test/0.0.7

group: 819066
lastModified: 2025-08-19T02:23:37Z
mimeType: application/zip
name: agnostic-app-test.zip
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/agnostic-app-test/0.0.7/agnostic-app-test.zip
size: 780
type: file
url: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.7/agnostic-app-test.zip


group: 819066
lastModified: 2025-08-19T02:23:37Z
mimeType: application/json
name: app.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/agnostic-app-test/0.0.7/app.json
size: 4243
type: file
url: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.7/app.json


group: 819066
lastModified: 2025-08-19T02:23:37Z
mimeType: application/json
name: profile.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/agnostic-app-test/0.0.7/profile.json
size: 379
type: file
url: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.

---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [23]:
if do_register:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List all apps

In [24]:
if do_register:
    listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
    select = 'id,created,description,version,owner' # Attributes to return in each result.
    orderBy = 'created(asc)'
    results = t.apps.getApps( orderBy=orderBy,
                             select=select)  
    for thisRes in results:
        print('--')
        print(thisRes)

--

created: 2024-12-13T20:15:33.014747Z
description: Run shell commands on remote systems
id: shell-runner-1.0.0
owner: silvia
version: 1.0.0
--

created: 2024-12-14T15:49:21.510206Z
description: Run shell commands on remote systems
id: shell-runner
owner: silvia
version: 1.0.0
--

created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
owner: silvia
version: latest
--

created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
owner: silvia
version: latest
--

created: 2025-08-17T22:34:01.193589Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-copy-silvia
owner: silvia
vers

#### List the new app

In [25]:
if do_register:
    appMetaData = t.apps.getAppLatestVersion(appId=app_def["id"])
    print(appMetaData)


containerImage: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.7/agnostic-app-test.zip
created: 2025-08-19T02:23:37.872268Z
deleted: False
description: Run any Executable
enabled: True
id: agnostic-app-test
isPublic: False
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: stampede3
cmdPrefix: None
coresPerNode: 48
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: stampede3
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: skx
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: [
autoMountLocal: True
description: Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP
envKey: inputDir

## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

In [26]:
print('app_id:',app_id)

app_id: agnostic-app-test


### App-Specific Input

In [27]:
# initalize
tapisInputAll = {}
tapisInputAll["name"] = f'test_{app_id}'

tapisInputAll["appId"] = app_id 
# tapisInputAll["appVersion"] = app_version # defaults to latest in description def
tapisInputAll["maxMinutes"] = 6

tapisInputAll["execSystemId"] = "stampede3" # the app runs on stampede only
tapisInputAll["execSystemLogicalQueue"] = "skx-dev" # "skx", "skx-dev"
tapisInputAll["nodeCount"] = 1 # limits set by which compute nodes you use
tapisInputAll["coresPerNode"] = 48 # limits set by which compute nodes you use
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='MyData' # Options: MyData or Work

### Job-Specific Input

In [28]:
# initalize
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'

tapisInput["name"] = tapisInput["name"] + '_' + tapisInput['storage_system']

tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 
# tapisInput['main_program'] = 'OpenSeesMP'
# tapisInput['input_filename'] = 'Ex1a_verymany.Canti2D.Push.mp.tcl'
# tapisInput['main_program'] = 'OpenSeesSP'
# tapisInput['input_filename'] = 'simpleSP.tcl'
tapisInput['main_program'] = 'python'
tapisInput['input_filename'] = 'Ex1a.Canti2D.Push.mpi.py'
tapisInput['command_line_args'] = '[33,22]'

In [29]:
job_description = OpsUtils.get_tapis_job_description(t, tapisInput)
print('job_description',job_description)

All Input is Complete
job_description {'name': 'test_agnostic-app-test_CommunityData', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.7', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'python'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.mpi.py'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'designsafe.storage.default', 'archiveSystemDir': '${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}'}


### Submit

In [30]:
# -----------------------------------------------------
if do_submitJob:
    jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    print(jobReturns.keys())
    print('JobStatusData:',jobReturns['JobStatusData'])
# -----------------------------------------------------

Creating job_description
All Input is Complete
Submitting Job
Job submitted! ID: 2849834c-2276-40d7-a752-79613b8a49d8-007
job_start_time: 1755570222.237398

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.09 sec	 Current Status: PENDING
	 Elapsed job time: 2.17 sec	 Current Status: PROCESSING_INPUTS		(PENDING took 1.07 sec)
	 Elapsed job time: 3.22 sec	 Current Status: STAGING_INPUTS		(PROCESSING_INPUTS took 1.05 sec)
	 Elapsed job time: 39.83 sec	 Current Status: STAGING_JOB		(STAGING_INPUTS took 36.61 sec)
	 Elapsed job time: 71.09 sec	 Current Status: SUBMITTING_JOB		(STAGING_JOB took 31.27 sec)
	 Elapsed job time: 72.17 sec	 Current Status: QUEUED		(SUBMITTING_JOB took 1.07 sec)
	 Elapsed job time: 74.23 sec	 Current Status: RUNNING		(QUEUED took 2.07 sec)
	 Elapsed job time: 128.84 sec	 Current Status: ARCHIVING		(RUNNING took 54.61 sec)
	 Elapsed job time: 159.18 sec	 Current Status: FINISHED		(ARCHIVING took 30.34 sec)
	  Status: FINISHED	 Elapsed job 

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (2849834c-2276-40d7-a752-79613b8a49d8-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (2849834c-2276-40d7-a752-79613b8a49d…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (2849834c-2276-40d7-a752-79613b8…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (2849834c-2276-40d7-a752-79613b8a49d…

dict_keys(['jobUuid', 'submitted_job', 'job_start_time', 'runJobStatus', 'job_description', 'JobHistory', 'JobMetadata', 'JobFiledata', 'JobStatusData'])
JobStatusData: None


## ✅ Summary

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

